In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/Image Retrieval'
!ls

/content/drive/MyDrive/Image Retrieval
 augmentation.ipynb
 binary_scenario
 binary_scenario_augmented_400X
 checkpoint
 citra_split.ipynb
 core
'data test splitting.ipynb'
 extract_feature_binary.ipynb
 extract_feature_subclass.ipynb
 index_images.ipynb
 original
 paper
 reconstruction
 retrieval_1_100X.ipynb
 retrieval_1_100X_subclass.ipynb
 retrieval_1_200X.ipynb
 retrieval_1_200X_subclass.ipynb
 retrieval_1_400X.ipynb
 retrieval_1_400X_subclass.ipynb
 retrieval.ipynb
 subclass_400
 subclass_scenario
 train_auto_encoder_magnification_1_100.ipynb
 train_auto_encoder_magnification_1_200.ipynb
 train_auto_encoder_magnification_1_400.ipynb
 train_auto_encoder_subclass_1_100.ipynb
 train_auto_encoder_subclass_1_200.ipynb
 train_auto_encoder_subclass_1_400.ipynb
 training_1_100
 training_1_100.h5
 training_1_100.json
 training_1_100_subclass
 training_1_100_subclass.h5
 training_1_100_subclass.json
 training_1_200
 training_1_200.h5
 training_1_200.json
 training_1_200_subclass
 training_

In [ ]:
import copy
import numpy as np
import os
import json
import cv2
import sklearn.metrics as metric
from core.AutoEncoder1 import ConvAutoEncoder
from tensorflow.keras.models import Model
import pandas as pd

In [ ]:
def euclidean(a, b):
	# compute and return the euclidean distance between two vectors
	return np.linalg.norm(a - b)

In [ ]:
def perform_search(query_features, indexed_train, max_results=5):
	retrieved = []
	for idx in range(0, len(indexed_train["features"])):
		distance = euclidean(query_features, indexed_train["features"][idx])
		retrieved.append((distance, idx))
	retrieved = sorted(retrieved)[:max_results]
	return retrieved

In [ ]:
base_dataset = "subclass_scenario"
magnification = "100X"
class_dir = ['tubular_adenoma', 'phyllodes_tumor', 'papillary_carcinoma',
             'mucinous_carcinoma', 'lobular_carcinoma', 'fibroadenoma',
             'ductal_carcinoma', 'adenosis']
IMAGE_SIZE = (256, 256)

In [ ]:
print("[INFO] indexing file images BreaKHis dataset...")
# indexing file images
dataset = []
for class_item in class_dir:
    cur_dir = os.path.join(base_dataset, 'test', magnification ,class_item)
    for file in os.listdir(cur_dir):
        dataset.append(os.path.join(cur_dir, file))

[INFO] indexing file images BreaKHis dataset...


In [ ]:
print("len to retrieving:", len(dataset))

len to retrieving: 207


In [ ]:
print("[INFO] load images BreaKHis dataset...")
#  load images
images = []
for image_path in dataset:
    if ".png" in image_path:
        image = cv2.imread(image_path)
        image = cv2.resize(image, IMAGE_SIZE)
        images.append(image)

[INFO] load images BreaKHis dataset...


In [ ]:
# normalization
print("[INFO] normalization...")
test_x = np.array(images).astype("float32") / 255.0

[INFO] normalization...


In [ ]:
auto_encoder = ConvAutoEncoder.build(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
# load our auto_encoder from disk
print("[INFO] loading auto encoder model...")
auto_encoder.load_weights("training_1_100_subclass/cp.ckpt")
with open('training_1_indexed_100_subclass.json') as f:
  training_indexed = json.load(f)

[INFO] loading auto encoder model...


In [ ]:
# create the encoder model which consists of *just* the encoder
# portion of the auto encoder
encoder = Model(inputs=auto_encoder.input,
	outputs=auto_encoder.get_layer("encoded").output)

# quantify the contents of our input images using the encoder
print("[INFO] encoding images...")
features_retrieved = encoder.predict(test_x)

[INFO] encoding images...


In [ ]:
query_indexes = list(range(0, test_x.shape[0]))
label_builder = list(np.unique(training_indexed["labels"]))
class_builder = {label_unique:[] for label_unique in label_builder}
recalls = copy.deepcopy(class_builder)
precisions = copy.deepcopy(class_builder)
# loop over the testing indexes
for i in query_indexes:
    queryFeatures = features_retrieved[i]
    results = perform_search(queryFeatures, training_indexed, max_results=5)
    labels_ret = [training_indexed["labels"][r[1]] for r in results]
    label_true = dataset[i].split("/")[3]
    label_trues = [label_true for _ in labels_ret]
    recall = metric.recall_score(label_trues, labels_ret, average='weighted')
    precision = metric.precision_score(label_trues, labels_ret, average='weighted')
    recalls[label_true].append(recall)
    precisions[label_true].append(precision)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("recall values:")
comb_recall, comb_precision = [], []
for key in recalls.keys():
    average_val = np.average(recalls[key])
    print(key, average_val)
    comb_recall.append(average_val)
print("combined recall", np.average(comb_recall))

print("\nprecision values:")
for key in precisions.keys():
    average_val = np.average(precisions[key])
    print(key, average_val)
    comb_precision.append(average_val)
print("combined precision", np.average(comb_precision))



recall values:
adenosis 0.12727272727272726
ductal_carcinoma 0.7088888888888889
fibroadenoma 0.3461538461538462
lobular_carcinoma 0.5647058823529413
mucinous_carcinoma 0.1272727272727273
papillary_carcinoma 0.14285714285714285
phyllodes_tumor 0.3
tubular_adenoma 0.26666666666666666
combined recall 0.32297723518311755

precision values:
adenosis 0.45454545454545453
ductal_carcinoma 0.9555555555555556
fibroadenoma 0.7307692307692307
lobular_carcinoma 0.9411764705882353
mucinous_carcinoma 0.4090909090909091
papillary_carcinoma 0.5714285714285714
phyllodes_tumor 0.8333333333333334
tubular_adenoma 0.5333333333333333
combined precision 0.678654107330578
